In [1]:
import pandas as pd
import regex as re
import matplotlib.pyplot as plt
import nltk
from datetime import datetime
import numpy as np

In [13]:
judgments = pd.read_csv(r"D:\Proyectos\amag\classification\judgments_and_orders\judgments_2018.csv")


In [14]:
judgments = judgments[~judgments['resolution_type'].str.contains('vista', case=False, na=False)]
judgments = judgments[judgments['law_field'].str.contains('familia', case=False, na=False)]
judgments['summary'] = judgments['summary'].str.strip()
judgments = judgments.reset_index()
judgments

,index,case_file,id,issued_date,resolution_number,resolution_type,notified_date,summary,court,judicial_district,judge,start_date,procedure_type,law_field,sub_law_field,opinion_text,judge_from_opinion
0,3,00002-2018-0-0201-JP-FC-02,documentoD.html?nid=IHlZsUeXRVbYJpF,2018-05-09,CINCO,SENTENCIA,05/09/2018,SENTENCIA FUNDADA EN PARTE LA DEMANDA INTERPUE...,2° JUZGADO PAZ LETRADO - Sede Central,ANCASH,SANTISTEBAN VALENZUELA AILLENY ADELA,2018-01-04,UNICO,FAMILIA CIVIL,ALIMENTOS,\n2° JUZGADO PAZ LETRADO - Sede Central \nEXP...,NaN
1,5,00006-2018-0-0210-JP-FC-01,documentoD.html?nid=GoktBpGEdOROVEm,2021-02-08,TRECE,SENTENCIA,24/08/2021,DECISIÓN:\nPOR TALES CONSIDERACIONES CON LA FA...,JUZGADO DE PAZ LETRADO - Sede Pomabamba,ANCASH,ALVAREZ ACERO ROCIO DELSY,2018-01-11,ESPECIAL,FAMILIA CIVIL,DECLARACION JUDICIAL DE PATERNIDAD EXTRAMATRIM...,JUZGADO DE PAZ LETRADO - Sede Pomabamba \nEXPE...,NaN
2,6,00006-2018-0-0215-JP-FC-01,documentoD.html?nid=JduMwGNHzBYHcRxVouD,2021-12-08,VEINTIOCHO,SENTENCIA,25/08/2021,FALLA: DECLARANDO FUNDADA EN PARTE LA DEMAN...,JUZGADO DE PAZ LETRADO ITINERANTE-Sede Asunción,ANCASH,HINOSTROZA HUERTA HILDA MARIA,2018-04-12,UNICO,FAMILIA CIVIL,AUMENTO DE ALIMENTOS,JUZGADO DE PAZ LETRADO ITINERANTE-Sede Asunció...,NaN
3,7,00007-2018-0-0201-JP-FC-02,documentoD.html?nid=yyHFwZbLpJQIWNYQJ,2020-10-30,DIECISIETE,SENTENCIA,30/10/2020,FALLA:\n\n1.\tDECLARANDO: INFUNDADA LA DEMANDA...,2° JUZGADO FAMILIA - Sede Central,ANCASH,VILLAFAN CANO HOMMER FREY,2018-01-05,UNICO,FAMILIA CIVIL,EXONERACION DE ALIMENTOS,CORTE SUPERIOR DE JUSTICIA DE ANCASH \n\nTERCE...,NaN
4,8,00008-2018-0-0201-JP-FC-01,documentoD.html?nid=IEsgCKwvAcFQGWpZh,2019-10-29,ONCE,SENTENCIA,29/10/2019,SENTENCIAS,3°JUZGADO DE PAZ LETRADO - Sede Central,ANCASH,"HUERTA BOJORQUEZ, CARMEN EUGENIA",2018-01-05,UNICO,FAMILIA CIVIL,ALIMENTOS,CORTE...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25616,65074,00626-2018-0-3301-JP-FC-01,documentoD.html?nid=BAAcYMBsLaFQCaNoJrJ,2019-06-21,CINCO,SENTENCIA,21/06/2019,SENTENCIA,2º JUZGADO DE PAZ LETRADO DE FAMILIA DE VENTAN...,PUENTE PIEDRA - VENTANILLA,RODRIGUEZ ROSALES CARLOS,2018-11-14,UNICO,FAMILIA CIVIL,ALIMENTOS,PODER JUDICIAL \n\nCORTE SUPERIOR DE JUSTICIA ...,NaN
25617,65075,00628-2018-0-3301-JP-FC-01,documentoD.html?nid=msMcoOcyAkiyLVjwf,2020-01-07,OCHO,SENTENCIA,01/07/2020,SENTENCIA- IMPROCEDENTE,2º JUZGADO DE PAZ LETRADO DE FAMILIA DE VENTAN...,PUENTE PIEDRA - VENTANILLA,RODRIGUEZ ROSALES CARLOS,2018-11-19,ESPECIAL,FAMILIA CIVIL,DECLARACION JUDICIAL DE PATERNIDAD EXTRAMATRIM...,\n\nPODER JUDICIAL \n\nCORTE SUPERIOR DE JUST...,NaN
25618,65184,00161-2018-1-3301-JR-FT-01,documentoD.html?nid=eDtOjjcTrAWwydLiQNV,2018-10-24,SEIS,SENTENCIA,24/10/2018,1)\tMANTENER VIGENTE LAS MEDIDAS DE PROTECCIÓN...,3° JUZGADO DE FAMILIA DE VENTANILLA Y MI PERÚ,PUENTE PIEDRA - VENTANILLA,YOLANDA PETRONILA CAMPOS SOTELO,2018-04-23,UNICO,FAMILIA TUTELAR,VIOLENCIA FAMILIAR,\n\nJUZGADO CIVIL - SEDE MI PERÚ\nEXPEDIENTE ...,YOLANDA PETRONILA CAMPOS SOTELO
25619,65192,00005-2018-0-3301-SP-FC-01,documentoD.html?nid=eIfLYIiqfylreNpGWu,2020-04-03,ONCE,SENTENCIA,05/03/2020,1.\tDECLARANDO FUNDADA LA SOLICITUD INTERPUEST...,SALA CIVIL,PUENTE PIEDRA - VENTANILLA,NaN,2018-06-15,CONOCIMIENTO,FAMILIA CIVIL,DIVORCIO POR CAUSAL,CORTE SUPERIOR DE JUSTICIA DE VENTANILLA Y \n\...,NaN


In [15]:
judgments

,index,case_file,id,issued_date,resolution_number,resolution_type,notified_date,summary,court,judicial_district,judge,start_date,procedure_type,law_field,sub_law_field,opinion_text,judge_from_opinion
0,3,00002-2018-0-0201-JP-FC-02,documentoD.html?nid=IHlZsUeXRVbYJpF,2018-05-09,CINCO,SENTENCIA,05/09/2018,SENTENCIA FUNDADA EN PARTE LA DEMANDA INTERPUE...,2° JUZGADO PAZ LETRADO - Sede Central,ANCASH,SANTISTEBAN VALENZUELA AILLENY ADELA,2018-01-04,UNICO,FAMILIA CIVIL,ALIMENTOS,\n2° JUZGADO PAZ LETRADO - Sede Central \nEXP...,NaN
1,5,00006-2018-0-0210-JP-FC-01,documentoD.html?nid=GoktBpGEdOROVEm,2021-02-08,TRECE,SENTENCIA,24/08/2021,DECISIÓN:\nPOR TALES CONSIDERACIONES CON LA FA...,JUZGADO DE PAZ LETRADO - Sede Pomabamba,ANCASH,ALVAREZ ACERO ROCIO DELSY,2018-01-11,ESPECIAL,FAMILIA CIVIL,DECLARACION JUDICIAL DE PATERNIDAD EXTRAMATRIM...,JUZGADO DE PAZ LETRADO - Sede Pomabamba \nEXPE...,NaN
2,6,00006-2018-0-0215-JP-FC-01,documentoD.html?nid=JduMwGNHzBYHcRxVouD,2021-12-08,VEINTIOCHO,SENTENCIA,25/08/2021,FALLA: DECLARANDO FUNDADA EN PARTE LA DEMAN...,JUZGADO DE PAZ LETRADO ITINERANTE-Sede Asunción,ANCASH,HINOSTROZA HUERTA HILDA MARIA,2018-04-12,UNICO,FAMILIA CIVIL,AUMENTO DE ALIMENTOS,JUZGADO DE PAZ LETRADO ITINERANTE-Sede Asunció...,NaN
3,7,00007-2018-0-0201-JP-FC-02,documentoD.html?nid=yyHFwZbLpJQIWNYQJ,2020-10-30,DIECISIETE,SENTENCIA,30/10/2020,FALLA:\n\n1.\tDECLARANDO: INFUNDADA LA DEMANDA...,2° JUZGADO FAMILIA - Sede Central,ANCASH,VILLAFAN CANO HOMMER FREY,2018-01-05,UNICO,FAMILIA CIVIL,EXONERACION DE ALIMENTOS,CORTE SUPERIOR DE JUSTICIA DE ANCASH \n\nTERCE...,NaN
4,8,00008-2018-0-0201-JP-FC-01,documentoD.html?nid=IEsgCKwvAcFQGWpZh,2019-10-29,ONCE,SENTENCIA,29/10/2019,SENTENCIAS,3°JUZGADO DE PAZ LETRADO - Sede Central,ANCASH,"HUERTA BOJORQUEZ, CARMEN EUGENIA",2018-01-05,UNICO,FAMILIA CIVIL,ALIMENTOS,CORTE...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25616,65074,00626-2018-0-3301-JP-FC-01,documentoD.html?nid=BAAcYMBsLaFQCaNoJrJ,2019-06-21,CINCO,SENTENCIA,21/06/2019,SENTENCIA,2º JUZGADO DE PAZ LETRADO DE FAMILIA DE VENTAN...,PUENTE PIEDRA - VENTANILLA,RODRIGUEZ ROSALES CARLOS,2018-11-14,UNICO,FAMILIA CIVIL,ALIMENTOS,PODER JUDICIAL \n\nCORTE SUPERIOR DE JUSTICIA ...,NaN
25617,65075,00628-2018-0-3301-JP-FC-01,documentoD.html?nid=msMcoOcyAkiyLVjwf,2020-01-07,OCHO,SENTENCIA,01/07/2020,SENTENCIA- IMPROCEDENTE,2º JUZGADO DE PAZ LETRADO DE FAMILIA DE VENTAN...,PUENTE PIEDRA - VENTANILLA,RODRIGUEZ ROSALES CARLOS,2018-11-19,ESPECIAL,FAMILIA CIVIL,DECLARACION JUDICIAL DE PATERNIDAD EXTRAMATRIM...,\n\nPODER JUDICIAL \n\nCORTE SUPERIOR DE JUST...,NaN
25618,65184,00161-2018-1-3301-JR-FT-01,documentoD.html?nid=eDtOjjcTrAWwydLiQNV,2018-10-24,SEIS,SENTENCIA,24/10/2018,1)\tMANTENER VIGENTE LAS MEDIDAS DE PROTECCIÓN...,3° JUZGADO DE FAMILIA DE VENTANILLA Y MI PERÚ,PUENTE PIEDRA - VENTANILLA,YOLANDA PETRONILA CAMPOS SOTELO,2018-04-23,UNICO,FAMILIA TUTELAR,VIOLENCIA FAMILIAR,\n\nJUZGADO CIVIL - SEDE MI PERÚ\nEXPEDIENTE ...,YOLANDA PETRONILA CAMPOS SOTELO
25619,65192,00005-2018-0-3301-SP-FC-01,documentoD.html?nid=eIfLYIiqfylreNpGWu,2020-04-03,ONCE,SENTENCIA,05/03/2020,1.\tDECLARANDO FUNDADA LA SOLICITUD INTERPUEST...,SALA CIVIL,PUENTE PIEDRA - VENTANILLA,NaN,2018-06-15,CONOCIMIENTO,FAMILIA CIVIL,DIVORCIO POR CAUSAL,CORTE SUPERIOR DE JUSTICIA DE VENTANILLA Y \n\...,NaN


In [18]:
cell_content = judgments.at[1289, 'summary']
cell_content

'1.\tFUNDADA LA DEMANDA INTERPUESTA POR DORA ISABEL QUISPE SUAREZ, EN REPRESENTACIÓN DE SU HIJO MIGUEL REVILLA QUISPE, CONTRA ELMER REVILLA MENDOZA, SOBRE FILIACIÓN JUDICIAL DE PATERNIDAD EXTRAMATRIMONIAL, EN LA VÍA DEL PROCESO ESPECIAL REGULADO EN LA LEY N° 28457, MODIFICADO POR LA LEY N° 30628; EN CONSECUENCIA: DECLARO QUE EL MANDATO JUDICIAL CONTENIDO EN LA RESOLUCIÓN N° 01 DE FOLIOS 10-11, SE CONVIERTA EN DECLARACIÓN JUDICIAL DE FILIACIÓN DE PATERNINAD EXTRAMATRIMONIAL DEL NIÑO MIGUEL REVILLA QUISPE; Y POR TANTO, DECLARO QUE EL PADRE DEL INDICADO MENOR ES EL DEMANDADO ELMER REVILLA MENDOZA. ORDENO UNA VEZ QUE QUEDE CONSENTIDA O EJECUTORIADA LA PRESENTE SENTENCIA, SE REMITAN LOS PARTES CORRESPONDIENTES, PARA QUE PROCEDA A INSCRIBIR LA FILIACIÓN DECLARADA, EN EL ACTA DE NACIMIENTO DEL NIÑO MIGUEL REVILLA QUISPE, NACIDO EL 16 DE MARZO DEL 2017. \n\t\n2.\tFUNDADA EN PARTE LA DEMANDA INTERPUESTA POR DORA ISABEL QUISPE SUAREZ, EN REPRESENTACIÓN DE SU HIJO MIGUEL REVILLA QUISPE, CONTRA EL

In [19]:
import openai
from openai import OpenAI
client = OpenAI(api_key=")

def generate_output(text):
    try:
        # Create chat completion
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Vas a clasificar el siguiente texto que contiene información jurídica. Es una tarea de cuatro clases. La clase 'Favorable' implica que la decisión judicial ha sido favorable para el demandante. Es favorable al demandante cuando la sentencia señala que la demanda es FUNDADA o cuando se  mantienen las medidas de protección. La clase 'Parcial' implica que la decisión judicial ha sido paracialmente favorable para el demandante. Es parcialmente favorable al demandante cuando la setencia es FUNDADA EN PARTE.  La clase 'Desfavorable' implica que es favorable para el demandado. Es favorable para el demandado cuando la sentencia es declarada 'INFUNDADA' o 'IMPROCEDENTE'. La cuarta clase es 'No precisa'. Esta clase debes usarla cuando no te es posible identificar un texto legal que contenga una decisión judicial."},
                {"role": "user", "content": text}
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")

judgments['outcome'] = judgments['summary'].apply(generate_output)

judgments.to_csv(r'D:\Proyectos\amag\classification\judgments_and_orders\jairo_demo.csv', index=False)


KeyboardInterrupt: 

In [20]:
judgments['outcome'] = judgments['summary'].apply(generate_output)

judgments.to_csv(r'D:\Proyectos\amag\classification\judgments_and_orders\jairo_demo.csv', index=False)

KeyboardInterrupt: 

In [21]:
judgments

,index,case_file,id,issued_date,resolution_number,resolution_type,notified_date,summary,court,judicial_district,judge,start_date,procedure_type,law_field,sub_law_field,opinion_text,judge_from_opinion
0,3,00002-2018-0-0201-JP-FC-02,documentoD.html?nid=IHlZsUeXRVbYJpF,2018-05-09,CINCO,SENTENCIA,05/09/2018,SENTENCIA FUNDADA EN PARTE LA DEMANDA INTERPUE...,2° JUZGADO PAZ LETRADO - Sede Central,ANCASH,SANTISTEBAN VALENZUELA AILLENY ADELA,2018-01-04,UNICO,FAMILIA CIVIL,ALIMENTOS,\n2° JUZGADO PAZ LETRADO - Sede Central \nEXP...,NaN
1,5,00006-2018-0-0210-JP-FC-01,documentoD.html?nid=GoktBpGEdOROVEm,2021-02-08,TRECE,SENTENCIA,24/08/2021,DECISIÓN:\nPOR TALES CONSIDERACIONES CON LA FA...,JUZGADO DE PAZ LETRADO - Sede Pomabamba,ANCASH,ALVAREZ ACERO ROCIO DELSY,2018-01-11,ESPECIAL,FAMILIA CIVIL,DECLARACION JUDICIAL DE PATERNIDAD EXTRAMATRIM...,JUZGADO DE PAZ LETRADO - Sede Pomabamba \nEXPE...,NaN
2,6,00006-2018-0-0215-JP-FC-01,documentoD.html?nid=JduMwGNHzBYHcRxVouD,2021-12-08,VEINTIOCHO,SENTENCIA,25/08/2021,FALLA: DECLARANDO FUNDADA EN PARTE LA DEMAN...,JUZGADO DE PAZ LETRADO ITINERANTE-Sede Asunción,ANCASH,HINOSTROZA HUERTA HILDA MARIA,2018-04-12,UNICO,FAMILIA CIVIL,AUMENTO DE ALIMENTOS,JUZGADO DE PAZ LETRADO ITINERANTE-Sede Asunció...,NaN
3,7,00007-2018-0-0201-JP-FC-02,documentoD.html?nid=yyHFwZbLpJQIWNYQJ,2020-10-30,DIECISIETE,SENTENCIA,30/10/2020,FALLA:\n\n1.\tDECLARANDO: INFUNDADA LA DEMANDA...,2° JUZGADO FAMILIA - Sede Central,ANCASH,VILLAFAN CANO HOMMER FREY,2018-01-05,UNICO,FAMILIA CIVIL,EXONERACION DE ALIMENTOS,CORTE SUPERIOR DE JUSTICIA DE ANCASH \n\nTERCE...,NaN
4,8,00008-2018-0-0201-JP-FC-01,documentoD.html?nid=IEsgCKwvAcFQGWpZh,2019-10-29,ONCE,SENTENCIA,29/10/2019,SENTENCIAS,3°JUZGADO DE PAZ LETRADO - Sede Central,ANCASH,"HUERTA BOJORQUEZ, CARMEN EUGENIA",2018-01-05,UNICO,FAMILIA CIVIL,ALIMENTOS,CORTE...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25616,65074,00626-2018-0-3301-JP-FC-01,documentoD.html?nid=BAAcYMBsLaFQCaNoJrJ,2019-06-21,CINCO,SENTENCIA,21/06/2019,SENTENCIA,2º JUZGADO DE PAZ LETRADO DE FAMILIA DE VENTAN...,PUENTE PIEDRA - VENTANILLA,RODRIGUEZ ROSALES CARLOS,2018-11-14,UNICO,FAMILIA CIVIL,ALIMENTOS,PODER JUDICIAL \n\nCORTE SUPERIOR DE JUSTICIA ...,NaN
25617,65075,00628-2018-0-3301-JP-FC-01,documentoD.html?nid=msMcoOcyAkiyLVjwf,2020-01-07,OCHO,SENTENCIA,01/07/2020,SENTENCIA- IMPROCEDENTE,2º JUZGADO DE PAZ LETRADO DE FAMILIA DE VENTAN...,PUENTE PIEDRA - VENTANILLA,RODRIGUEZ ROSALES CARLOS,2018-11-19,ESPECIAL,FAMILIA CIVIL,DECLARACION JUDICIAL DE PATERNIDAD EXTRAMATRIM...,\n\nPODER JUDICIAL \n\nCORTE SUPERIOR DE JUST...,NaN
25618,65184,00161-2018-1-3301-JR-FT-01,documentoD.html?nid=eDtOjjcTrAWwydLiQNV,2018-10-24,SEIS,SENTENCIA,24/10/2018,1)\tMANTENER VIGENTE LAS MEDIDAS DE PROTECCIÓN...,3° JUZGADO DE FAMILIA DE VENTANILLA Y MI PERÚ,PUENTE PIEDRA - VENTANILLA,YOLANDA PETRONILA CAMPOS SOTELO,2018-04-23,UNICO,FAMILIA TUTELAR,VIOLENCIA FAMILIAR,\n\nJUZGADO CIVIL - SEDE MI PERÚ\nEXPEDIENTE ...,YOLANDA PETRONILA CAMPOS SOTELO
25619,65192,00005-2018-0-3301-SP-FC-01,documentoD.html?nid=eIfLYIiqfylreNpGWu,2020-04-03,ONCE,SENTENCIA,05/03/2020,1.\tDECLARANDO FUNDADA LA SOLICITUD INTERPUEST...,SALA CIVIL,PUENTE PIEDRA - VENTANILLA,NaN,2018-06-15,CONOCIMIENTO,FAMILIA CIVIL,DIVORCIO POR CAUSAL,CORTE SUPERIOR DE JUSTICIA DE VENTANILLA Y \n\...,NaN


In [ ]:
judgments.to_csv(r'D:\Proyectos\amag\classification\judgments_and_orders\jairo_demo.csv', index=False)